In [1]:
import numpy as np
from models.linear import LinearRegression
from models.logistic import LogisticRegression
from metrics.classification_metrics import *

In [2]:
n_samples = 500
np.random.seed(42)

classes = ["A", "B", "C", "D"]

random_classes = np.random.choice(classes, size=n_samples)


class_to_int = {label: idx for idx, label in enumerate(classes)}
random_classes_int = np.vectorize(class_to_int.get)(random_classes)

In [3]:
X = np.random.normal(2, 2, n_samples)
Z = np.random.normal(9, 1, n_samples) 
P = np.random.beta(1, 2, n_samples)

Y = 2*X +  np.random.normal(0, 0.5, n_samples) + P + Z + random_classes_int * X 

X= np.column_stack((X,X**2, Z + X, X * P, P, random_classes_int))

In [4]:
reg = LinearRegression(regularization="None").fit(X,Y)
ridge = LinearRegression(regularization="Ridge").fit(X,Y)
lasso = LinearRegression(regularization="Lasso", n_iter = 10000, lr=0.001).fit(X,Y)
elastic = LinearRegression(regularization="ElasticNet", n_iter = 10000, lr=0.001, alpha=0.4).fit(X,Y)

In [5]:
elastic_pred = elastic.predict(X)
elastic.print_errors(Y, elastic_pred), lasso.print_errors(Y, lasso.predict(X)), ridge.print_errors(Y, ridge.predict(X))
elastic.score(X, Y), lasso.score(X, Y), ridge.score(X, Y)

╒══════════╤═════════╕
│ Metric   │   Value │
╞══════════╪═════════╡
│ MAE      │  1.9092 │
├──────────┼─────────┤
│ RMSE     │  2.753  │
├──────────┼─────────┤
│ MSE      │  7.5791 │
╘══════════╧═════════╛
╒══════════╤═════════╕
│ Metric   │   Value │
╞══════════╪═════════╡
│ MAE      │  1.7339 │
├──────────┼─────────┤
│ RMSE     │  2.4169 │
├──────────┼─────────┤
│ MSE      │  5.8413 │
╘══════════╧═════════╛
╒══════════╤═════════╕
│ Metric   │   Value │
╞══════════╪═════════╡
│ MAE      │  1.721  │
├──────────┼─────────┤
│ RMSE     │  2.3877 │
├──────────┼─────────┤
│ MSE      │  5.7009 │
╘══════════╧═════════╛


(Array(0.87876916, dtype=float32),
 Array(0.90656686, dtype=float32),
 np.float64(0.9088114287261543))

In [6]:
#elastic.do_all(X, Y)

In [7]:
#reg.Cooks_distance(X,Y)

In [8]:
classes = ["A", "B"]
n_samples = 1500

random_classes = np.random.choice(classes, size=n_samples)


class_to_int = {label: idx for idx, label in enumerate(classes)}
random_classes_int = np.vectorize(class_to_int.get)(random_classes)

X = np.random.normal(2, 2, n_samples)
Z = np.random.normal(9, 1, n_samples) 
P = np.random.beta(1, 2, n_samples)


X= np.column_stack((X, Z))

Y = random_classes_int


In [9]:
import pandas as pd

#X = pd.DataFrame(X, columns=["1","2","3","4","5"])

In [10]:
log = LogisticRegression(X.shape[1], len(classes), regularization="None")
log.fit(X, Y)

In [11]:
pred = log.predict(X)
pred

array([0, 1, 1, ..., 0, 0, 1], shape=(1500,))

In [12]:
confusion_matrix(Y, pred)
precision(Y, pred), recall(Y, pred), f1_stat(Y, pred), accuracy(Y, pred), balanced_accuracy(Y, pred), NPV(Y, pred), FOR(Y, pred)

(array([0.51041667, 0.50943396]),
 array([0.58565737, 0.43373494]),
 array([0.54545455, 0.46854664]),
 array(0.51),
 array(0.50969616),
 array([0.50943396, 0.51041667]),
 array([0.49056604, 0.48958333]))

In [13]:
data = []
for i in np.unique(Y):
    data.append((Y == i).astype(int))

In [14]:
data

[array([0, 1, 1, ..., 0, 0, 1], shape=(1500,)),
 array([1, 0, 0, ..., 1, 1, 0], shape=(1500,))]

In [15]:
from models.onevsall import OVA
ova = OVA(LogisticRegression(X.shape[1], len(classes), regularization="l1", n_iter=250))
ova.fit(X, Y)
pred = ova.predict(X)
confusion_matrix(Y, pred=pred), balanced_accuracy(Y, pred)

(array([[753.,   0.],
        [747.,   0.]]),
 array(0.5))

In [28]:
from models.onevsone import OVO
ovo = OVO(LogisticRegression(X.shape[1], len(classes), regularization="l2", n_iter=250))
ovo.fit(X, Y)
pred = ovo.predict(X)
confusion_matrix(Y, pred=pred), balanced_accuracy(Y, pred)

(array([[500., 253.],
        [484., 263.]]),
 array(0.5080428))

In [30]:
from models.svm import SVMClassificator

svm = SVMClassificator(n_iter=2000, C = 1, lr = 0.2)
svm.fit(X, Y)
preds = svm.predict(X) 
confusion_matrix(Y, pred=preds), balanced_accuracy(Y, preds)

(array([[  0., 753.],
        [  0., 747.]]),
 array(0.5))